In this assignment, you will implement scaled dot product attention, a key component to the transformer neural network architecture that has revolutionized natural language processing.

In [ ]:
!pip uninstall -y transformers huggingface_hub
!pip install -q --upgrade transformers huggingface_hub seaborn
# DON'T CHANGE ANYTHING IN THIS CELL

In [ ]:
import torch, torch.nn as nn, torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# load BERT base
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
bert = AutoModel.from_pretrained(
    "bert-base-uncased",
    attn_implementation="eager",
    output_attentions=True
)
# DON'T CHANGE ANYTHING IN THIS CELL

In [ ]:
def scaled_dot_product_attention(Q, K, V, mask=None):
    """
    Implements scaled-dot-product attention.
    Shapes:
        Q, K, V : (B, H, L, d_k)   # B=batch, H=heads, L=sequence
        mask    : broadcastable to (B, H, L, L)   (0 = pad, 1 = keep)
    Returns:
        out  : (B, H, L, d_k)
        attn : (B, H, L, L)
    """

    # ### TODO 1. raw scores = Q * Kᵀ (hint: transpose(-2, -1))
    scores =

    # ### TODO 2. scale by √d_k
    d_k = Q.size(-1)
    scores =

    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)

    # ### TODO 3. softmax over keys dimension (-1)
    attn =

    # ### TODO 4. multiply by V
    out =

    return out, attn


In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model=768, num_heads=8):
        super().__init__()
        assert d_model % num_heads == 0, "d_model must split evenly"
        self.h   = num_heads
        self.d_k = d_model // num_heads

        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)

    def forward(self, x, mask=None):
        B, L, C = x.shape

        # ### TODO 1-3. project & reshape for heads
        Q = self.W_q(x).view(B, L, self.h, self.d_k).transpose(1, 2)
        K =
        V =

        # ### TODO 4. call single-head attention
        out, attn =

        # ### TODO 5-6. merge heads & final linear
        out = out.transpose(1, 2).contiguous().view(B, L, C)
        out =

        return out, attn


In [ ]:
@torch.no_grad()
def embed_sentence(text):
    ids   = tokenizer(text, return_tensors="pt")
    emb   = bert.embeddings(ids["input_ids"])
    mask  = ids["attention_mask"].unsqueeze(1).unsqueeze(2)
    tokens = tokenizer.convert_ids_to_tokens(ids["input_ids"][0])
    return emb, mask, tokens

def plot_heads(attn, tokens, heads_per_row=4):
    """
    attn: (B=1, H, L, L)   attention from our MHA
    """
    attn = attn[0].cpu()
    H, L, _ = attn.shape
    rows = (H + heads_per_row - 1)//heads_per_row
    fig, axes = plt.subplots(rows, heads_per_row, figsize=(4*heads_per_row,4*rows))
    axes = axes.flatten()
    for h in range(H):
        sns.heatmap(attn[h], vmin=0, vmax=attn.max(),
                    cmap="viridis", square=True,
                    xticklabels=tokens, yticklabels=tokens,
                    ax=axes[h], cbar=False)
        axes[h].set_title(f"Head {h}")
        axes[h].tick_params(labelsize=8, rotation=90)
    for ax in axes[H:]:
        ax.axis("off")
    plt.tight_layout()
    plt.show()
    # DON'T CHANGE ANYTHING IN THIS CELL

In [ ]:
# Check our implementation (it isn't trained so the attention will be all over the place.)

sentence = "The quick brown fox jumps over the lazy dog." # ### TODO: Use a unique sentence!
x, mask, toks = embed_sentence(sentence)

mha = MultiHeadAttention(d_model=768, num_heads=12)
mha.eval()

with torch.no_grad():
    out, attn = mha(x, mask)

plot_heads(attn, toks)

In [ ]:
# See attention directly using BERT.

x, mask, toks = embed_sentence(sentence)

with torch.no_grad():
    outputs = bert(**tokenizer(sentence, return_tensors="pt"), output_attentions=True)
    attn = outputs.attentions[0]
plot_heads(attn, toks, heads_per_row=4)

In [ ]:
# Use pretrained BERT weights to test our implementation.

bert_sa   = bert.encoder.layer[0].attention.self
bert_dout = bert.encoder.layer[0].attention.output.dense

mha_pt = MultiHeadAttention(d_model=768, num_heads=12)

with torch.no_grad():
    mha_pt.W_q.weight.copy_(bert_sa.query.weight)
    mha_pt.W_q.bias.copy_( bert_sa.query.bias )
    mha_pt.W_k.weight.copy_(bert_sa.key.weight)
    mha_pt.W_k.bias.copy_( bert_sa.key.bias )
    mha_pt.W_v.weight.copy_(bert_sa.value.weight)
    mha_pt.W_v.bias.copy_( bert_sa.value.bias )
    mha_pt.W_o.weight.copy_(bert_dout.weight)
    mha_pt.W_o.bias.copy_( bert_dout.bias )

mha_pt.eval()
with torch.no_grad():
    out_pt, attn_pt = mha_pt(x, mask)
plot_heads(attn_pt, toks)

In [ ]:
# Visualize a different layer with layer_id.

layer_id   = 11 # TODO: Visualize different layers!
layer      = bert.encoder.layer[layer_id]

bert_sa    = layer.attention.self
bert_dout  = layer.attention.output.dense
mha_L      = MultiHeadAttention(d_model=768, num_heads=12)

with torch.no_grad():
    mha_L.W_q.weight.copy_(bert_sa.query.weight);  mha_L.W_q.bias.copy_(bert_sa.query.bias)
    mha_L.W_k.weight.copy_(bert_sa.key.weight);    mha_L.W_k.bias.copy_(bert_sa.key.bias)
    mha_L.W_v.weight.copy_(bert_sa.value.weight);  mha_L.W_v.bias.copy_(bert_sa.value.bias)
    mha_L.W_o.weight.copy_(bert_dout.weight);      mha_L.W_o.bias.copy_(bert_dout.bias)

mha_L.eval()

with torch.no_grad():
    x, mask, toks = embed_sentence(sentence)
    _, attn_L     = mha_L(x, mask)

plot_heads(attn_L, toks, heads_per_row=4)

If you are interested, here is an assignment to help you use pretrained transformer networks on next text generation, sentiment analysis, and masked token prediction. This is not required: https://colab.research.google.com/drive/1WvWL9ZxYd1PgJvMFdfT0Ye389TlQFrzA?usp=sharing